# Systeme de Recommandation V0
# Version 1 User



In [3]:

import sys
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joans\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joans\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\joans\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\joans\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
#recuperer la requete de filtrage et lutilsateur

user=sys.argv[1]
requete=sys.argv[2]


In [6]:
#Pour recuperer la listes des films faut avant recuperer tous les participants de la salle (liste de id de utilisateurs)
historique = "select id_show from regarder where id_us = " + str(user) + ";"

#recuperer la liste des films dans l'historique de l'utilisateur1(liste de id)
#recuperer la liste des films dans l'historique de l'utilisateur2(liste de id)
#recuperer la liste des films dans l'historique de l'utilisateur3(liste de id)
#recuperer la liste des films dans l'historique de l'utilisateur4(liste de id)


In [7]:

import pymysql
import pandas as pd
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
cnxn = pymysql.connect(user="nchoice", password="jenesaispas", host="localhost", database="nchoice")

print(cnxn)

In [8]:

#t = historique['id_show'].tolist()
t=("s3","s8","s10") #TEST
requete_historique = "SELECT show_new.*, GROUP_CONCAT(DISTINCT jouer.id_cast SEPARATOR ',') as cast_ids, GROUP_CONCAT(DISTINCT produire.id_direc SEPARATOR ',') as director_ids,GROUP_CONCAT(DISTINCT etre.id_genre SEPARATOR ',') as genre_ids FROM show_new LEFT JOIN jouer ON jouer.id_show = show_new.id_show LEFT JOIN produire ON produire.id_show = show_new.id_show LEFT JOIN etre ON etre.id_show = show_new.id_show  WHERE show_new.id_show IN {} GROUP BY show_new.id_show; ".format(t)

films_historiques = pd.read_sql(requete_historique, cnxn)
#films_filtres = pd.read_sql(requete, cnxn)

c:\Users\joans\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
# for every film in the list of films that passed the manual filtering, get the correlation between the film and the films in the historial (1 for user) (4 max) (id)
# faire la moyenne donc chaque user il aura un correspondance avec chaque film qui passe le filtre manuel

In [92]:
def list_similarity(list1, list2):
    if len(list1) == 0 or len(list2) == 0:
        return 0
    set1 = set(map(tuple, list1))
    set2 = set(map(tuple, list2))

    intersection = set1.intersection(set2)
    union = set1.union(set2)

    similarity = len(intersection) / len(union) 
    return similarity


In [123]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Remove stop words
    stop_words = set(stopwords.words("english"))
    filtered_text = [word for word in text.lower().split() if word not in stop_words]
    
    # Perform stemming
    ps = PorterStemmer()
    stemmed_text = [ps.stem(word) for word in filtered_text]
    
    # Perform lemmatization
    lem = WordNetLemmatizer()
    lemmatized_text = [lem.lemmatize(word, "v") for word in stemmed_text]
    
    return " ".join(lemmatized_text)

def calculate_similarity(filtres, historique):
    genre_weight = 0.2
    director_weight = 0.15
    cast_weight = 0.20
    type_weight = 0.05
    description_weight = 0.3
    year_weight = 0.05
    country_weight = 0.05
    # Create a copy of the dataframes
    filtres_similarity = filtres.copy()
    historique_similarity = historique.copy()
    
    #Preprocess the  lists columns
    filtres_similarity["cast_ids"]=filtres_similarity["cast_ids"].str.split(",")
    filtres_similarity["director_ids"]=filtres_similarity["director_ids"].str.split(",")
    filtres_similarity["genre_ids"]=filtres_similarity["genre_ids"].str.split(",")
    filtres_similarity["country"]=filtres_similarity["country"].str.split(",")

    historique_similarity["cast_ids"]=historique_similarity["cast_ids"].str.split(",")
    historique_similarity["director_ids"]=historique_similarity["director_ids"].str.split(",")
    historique_similarity["genre_ids"]=historique_similarity["genre_ids"].str.split(",")
    historique_similarity["country"]=historique_similarity["country"].str.split(",")

    # Preprocess the description column
    filtres_similarity['description'] = filtres_similarity['description'].apply(preprocess_text)
    historique_similarity['description'] = historique_similarity['description'].apply(preprocess_text)

    #Acces rows of the filtered films dataframe and calculate the similarity between every filtered film and the films in the historial
    df_similarity = pd.DataFrame(columns=['id_show', 'similarity'])
    for index, row in filtres_similarity.iterrows():
        similarity=0
        for index2, row2 in historique_similarity.iterrows():
            if row['type']==row2['type']:
                similarity+=1*type_weight
            if row['year']==row2['year']:
                similarity+=1*year_weight
            similarity+=list_similarity(row['genre_ids'],row2['genre_ids'])*genre_weight
            similarity+=list_similarity(row['director_ids'],row2['director_ids'])*director_weight
            similarity+=list_similarity(row['cast_ids'],row2['cast_ids'])*cast_weight
            similarity+=list_similarity(row['country'],row2['country'])*country_weight
            vectorizer = CountVectorizer()
            # Transform the descriptions into vectors
            vectors = vectorizer.fit_transform([row["description"], row2["description"]])
            # Calculate the cosine similarity between the two vectors
            cosine_sim = cosine_similarity(vectors[0], vectors[1])[0][0]
            similarity+=cosine_sim*description_weight

        new_row={'id_show':row['id_show'],'similarity':similarity/len(historique_similarity)}
        df_similarity=df_similarity.append(new_row, ignore_index=True)

   

    return df_similarity.sort_values(by=['similarity'], ascending=False)

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [124]:
#TEST
t=("s9","s812","s1010","s333")
requete= "SELECT show_new.*, GROUP_CONCAT(DISTINCT jouer.id_cast SEPARATOR ',') as cast_ids, GROUP_CONCAT(DISTINCT produire.id_direc SEPARATOR ',') as director_ids,GROUP_CONCAT(DISTINCT etre.id_genre SEPARATOR ',') as genre_ids FROM show_new LEFT JOIN jouer ON jouer.id_show = show_new.id_show LEFT JOIN produire ON produire.id_show = show_new.id_show LEFT JOIN etre ON etre.id_show = show_new.id_show  WHERE show_new.id_show IN {} GROUP BY show_new.id_show; ".format(t)
films_filtres = pd.read_sql(requete, cnxn)
films_filtres
films_filtres = films_filtres.astype(str)
films_historiques = films_historiques.astype(str)
calculate_similarity(films_filtres, films_historiques)


c:\Users\joans\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\joans\AppData\Local\Temp\ipykernel_3688\3096245369.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_similarity=df_similarity.append(new_row, ignore_index=True)
C:\Users\joans\AppData\Local\Temp\ipykernel_3688\3096245369.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_similarity=df_similarity.append(new_row, ignore_index=True)
C:\Users\joans\AppData\Local\Temp\ipykernel_3688\3096245369.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

,id_show,similarity
2,s812,0.083333
0,s1010,0.050000
3,s9,0.050000
1,s333,0.044048


# Version plusieurs utilisateurs